# Lokale Suche für Round-Robin-Tournaments

Der Spielplan in einer Sportliga lässt sich entweder als gerichteter Graph oder als Matrix darstellen.

Beispiel:

$$
S = 
\begin{pmatrix}
\phantom{-}4 &  \phantom{-}3 & \phantom{-}2 \\
\phantom{-}3 &  \phantom{-}4 &  -1 \\
-2 & -1 & -4 \\
-1 & -2 & \phantom{-}3
 \end{pmatrix}
$$

- Team 1 spielt am ersten Spieltag zuhause gegen Team 4.
- Team 4 spielt am ersten Spieltag auswärts gegen Team 1.

Aus jedem Spielplan lassen sich durch bestimmte Vertauschungen neue (gültige) Spielpläne erzeugen.


In [ ]:
%load_ext line_profiler

In [ ]:
import numpy as np

S = np.array([
       [ -3,  12,  -4,  11,   9,  -8,   6,  -7,  10,  -5,   2],
       [  9,  -8,   6,  -7,  10,  -5, -11,   3, -12,   4,  -1],
       [  1,  -9,   8,  -6,   7, -10,   5,  -2, -11,  12,  -4],
       [-12,  11,   1,  -9,   8,  -6,   7, -10,   5,  -2,   3],
       [  8,  -6,   7, -10, -11,   2,  -3,  12,  -4,   1,  -9],
       [-10,   5,  -2,   3, -12,   4,  -1,   9,  -8, -11,   7],
       [-11,  10,  -5,   2,  -3,  12,  -4,   1,  -9,   8,  -6],
       [ -5,   2,  -3,  12,  -4,   1,  -9,  11,   6,  -7,  10],
       [ -2,   3, -12,   4,  -1,  11,   8,  -6,   7, -10,   5],
       [  6,  -7, -11,   5,  -2,   3, -12,   4,  -1,   9,  -8],
       [  7,  -4,  10,  -1,   5,  -9,   2,  -8,   3,   6, -12],
       [  4,  -1,   9,  -8,   6,  -7,  10,  -5,   2,  -3,  11]])

In [ ]:
def PartialSwapRounds(S, i, s1, s2):
    """ Tauscht für Team i die Spieltage s1 und s2 und führt Reperaturvertauschungen durch,
    so dass der Spielplan zulässig bleibt """
    Sneu = np.copy(S)
    L = [i]
    K = [abs(S[i - 1, s1 - 1]), abs(S[i - 1, s2 - 1])]
    # Finde nun alle Zeilen, die getauscht werden müssen, damit es ein gültiges SRR bleibt
    # Die entsprechenden Zeilen werden in L gespeichert
    while len(K) != 0:
        kandidat = K[0]
        K.remove(kandidat)
        if kandidat not in L:
            L.append(kandidat)
        links = abs(S[kandidat - 1, s1 - 1])
        rechts = abs(S[kandidat - 1, s2 - 1])
        if links not in L and links not in K:
            K.append(links)
        if rechts not in L and rechts not in K:
            K.append(rechts)
    # Tausche nun die entsprechenden Zeilen
    for t in L:
        Sneu[t - 1, [s1 - 1, s2 - 1]] = S[t - 1, [s2 - 1, s1 - 1]]
    return Sneu

In [ ]:
S2 = PartialSwapRounds(S, 1, 1, 2)
S2

In [ ]:
%timeit PartialSwapRounds(S, 1, 1, 2)

In [ ]:
%lprun -f PartialSwapRounds PartialSwapRounds(S, 1, 1, 2)

# Cython

### Version 1

In [ ]:
%load_ext cython

In [ ]:
%%cython -a -f --cplus -c=-O3 -c=-march=native -c=-Wno-deprecated-declarations -c=-Wno-#warnings

from cython cimport wraparound, boundscheck
import numpy as np
cimport numpy as np

@wraparound(False)
@boundscheck(False)
cpdef PartialSwapRounds_cy1(long[:, ::1] Sin, int i, int s1, int s2):
    cdef long[:, ::1] S = Sin.copy()
    cdef int tmp, kandidat, links, rechts, t
    cdef list L = [i]
    cdef list K = [abs(S[i - 1, s1 - 1]), abs(S[i - 1, s2 - 1])]
    while len(K) != 0:
        kandidat = K[0]
        K.remove(kandidat)
        if kandidat not in L:
            L.append(kandidat)
        links = abs(S[kandidat - 1, s1 - 1])
        rechts = abs(S[kandidat - 1, s2 - 1])
        if links not in L and links not in K:
            K.append(links)
        if rechts not in L and rechts not in K:
            K.append(rechts)
    # Tausche nun die entsprechenden Zeilen
    for t in L:
        tmp = S[t-1,s1-1]
        S[t-1,s1-1] = S[t-1,s2-1]
        S[t-1,s2-1] = tmp
    return np.asarray(S)

In [ ]:
%timeit PartialSwapRounds_cy1(S, 1, 1, 2)

### Version 2

In [ ]:
%%cython -a -f --cplus -c=-O3 -c=-march=native -c=-Wno-deprecated-declarations -c=-Wno-#warnings

from libc.math cimport abs as abs
from libcpp.vector cimport vector
from libcpp.algorithm cimport find
from cython cimport wraparound, boundscheck
import numpy as np
cimport numpy as np

@wraparound(False)
@boundscheck(False)
cpdef PartialSwapRounds_cy2(long[:, ::1] Sin, int i, int s1, int s2):
    cdef long[:, ::1] S = Sin.copy()
    cdef vector[int] L = vector[int](1)
    cdef vector[int] K = vector[int](2)
    cdef int kandidat, links, rechts, t, tmp, N
    cdef unsigned int ii = 0
    
    L[0] = i
    K[0] = abs(S[i-1,s1-1])
    K[1] = abs(S[i-1,s2-1])
    
    while ii < K.size():
        kandidat = K[ii]
        # if kandidat not in L:
        if find(L.begin(), L.end(), kandidat) == L.end():
            L.push_back(kandidat)
        links = abs(S[kandidat - 1, s1 - 1])
        rechts = abs(S[kandidat - 1, s2 - 1])
        # if links not in L and links not in K
        if find(L.begin(), L.end(), links) == L.end() and find(K.begin(), K.end(), links) == K.end():
            K.push_back(links)
        # if rechts not in L and rechts not in K
        if find(L.begin(), L.end(), rechts) == L.end() and find(K.begin(), K.end(), rechts) == K.end():
            K.push_back(rechts)
        ii += 1
    # Tausche nun die entsprechenden Zeilen
    for t in L:
        tmp = S[t-1, s1-1]
        S[t-1,s1-1] = S[t-1,s2-1]
        S[t-1,s2-1] = tmp
    return np.asarray(S)

In [ ]:
%timeit PartialSwapRounds_cy2(S, 1, 1, 2)